In [12]:
import pandas as pd
import pytest
import mysql.connector
import pymysql
from dotenv import load_dotenv
from sqlalchemy import create_engine
import os
load_dotenv()

True

In [2]:
# Connect to the source and destination databases
source_db = create_engine(os.getenv('SOURCE_DB_CONN'))

destination_db = create_engine(os.getenv('DEST_DB_CONN'))


In [3]:
def client_data_load():
    # read data from source tables
    tablename = 'client'
    client_df = pd.read_sql_table(tablename, source_db)

    # mask PII
    pii_columns = ['email_address', 'name' , 'address', 'phone_number']
    client_df[pii_columns] = client_df[pii_columns].applymap(lambda x: x[0:2] + '*' * (len(x) -3) + x[-1:])

    #add insert timestamp
    client_df['insert_time'] = pd.Timestamp.now()

    # insert into destination database
    client_df.to_sql('client', con=destination_db, if_exists='replace', index=False)
    return client_df
# client_data_load()
print(f"{len(client_data_load())} rows from client table were inserted successfully at {str(pd.Timestamp.now())}")

100 rows from client table were inserted successfully at 2023-03-24 09:49:49.276642


In [7]:
#testing the pipeline
def test_client_data_load():
    source_table = 'client'
    destination_table = 'client'
    source_db = create_engine(os.getenv('SOURCE_DB_CONN'))
    destination_db = create_engine(os.getenv('DEST_DB_CONN'))
    
    dest_df= pd.read_sql_table(destination_table,destination_db)
    source_df=pd.read_sql_table(source_table,source_db)
    pii_columns = ['email_address', 'name' , 'address', 'phone_number']

    #check that table destination is not empty
    assert len(dest_df) >1
    # assert len(source_df) == len(dest_df) #check number of records match

    #check if pii columns are masked
    assert dest_df['email_address'].str.contains('\*\*').all() #check whether email is masked
    # assert dest_df['name'].str.contains('\*').all() #check whether name is masked
    assert dest_df['address'].str.contains('\*\*').all() #check whether address is masked
    assert dest_df['phone_number'].str.contains('\*\*').all() #check whether phone_number is masked

    return test_client_data_load()

In [16]:
def enrollee_data_load():
    
    tablename = 'enrollee_profile'
    enrollee_df = pd.read_sql_table(tablename, source_db)

    # mask PII
    pii_columns = ['hmo_id' , 'other_names', 'home_phone_number']
    enrollee_df[pii_columns] = enrollee_df[pii_columns].applymap(lambda x: x[0:2] + '*' * (len(x) -3) + x[-1:])

    #add insert timestamp
    enrollee_df['insert_time'] = pd.Timestamp.now()

    # insert into destination database
    enrollee_df.to_sql('enrollee_profile', con=destination_db, if_exists='replace', index=False)
    
    return enrollee_df
# enrollee_data_load()

print(f"{len(enrollee_data_load())} rows from client table were inserted successfully at {str(pd.Timestamp.now())}")
    


100 rows from client table were inserted successfully at 2023-03-24 09:55:46.537360


In [10]:
def test_enrollee_data_load():
    source_table = 'enrollee_profile'
    destination_table = 'enrollee_profile'
    source_db = create_engine(os.getenv('SOURCE_DB_CONN'))
    destination_db = create_engine(os.getenv('DEST_DB_CONN'))
    
    dest_df= pd.read_sql_table(destination_table,destination_db)
    source_df=pd.read_sql_table(source_table,source_db)
    pii_columns = ['email_address', 'name' , 'address', 'phone_number']

    #check that table destination is not empty
    assert len(dest_df) >1
    # assert len(source_df) == len(dest_df) #check number of records match

    #check if pii columns are masked
    assert dest_df['home_phone_number'].str.contains('\*\*').all() #check whether email is masked
    assert dest_df['hmo_id'].str.contains('\*').all() #check whether name is masked
    assert dest_df['home_phone_number'].str.contains('\*\*').all() #check whether address is masked
        

In [14]:
if __name__ == '__main__':
    pytest.main()

ERROR: usage: ipykernel_launcher.py [options] [file_or_dir] [file_or_dir] [...]
ipykernel_launcher.py: error: unrecognized arguments: -f
  inifile: None
  rootdir: /Users/reliancehmo



In [15]:
#close database connections
source_db.dispose()
destination_db.dispose()
print('Connection closed')


Connection closed
